# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
import networkx as nx
import gensim
import numpy as np
import multiprocessing as mp
from metricwalk.functions import generate_walks_for_node

class WireWalk():
    def __init__(self, graph: nx.Graph, dimensions: int = 128, window: int = 10, 
                walk_length: int = 80, num_walks: int = 10, 
                workers: int = 1):
        self.graph = graph
        self.dimensions = dimensions
        self.window = window
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.workers = workers
    
    def generate_transition(self, transition_matrix_function):
        G = self.graph
        nodes = G.nodes
        pool = mp.Pool()

        params = []
        for current_node in nodes:
            for destination in nodes:
                params.append((G,current_node, destination))

        ss_weights = pool.starmap(transition_matrix_function, params)
        # close the process pool
        pool.close()
        # wait for all tasks to complete
        pool.join()

        ss_weights_matrix1 = np.reshape(ss_weights, (len(nodes), len(nodes)))
        row_sums = ss_weights_matrix1.sum(axis=1)
        zero_rows = np.where(row_sums == 0)[0]
        ss_weights_matrix1[zero_rows, :] = 0
        for i in zero_rows:
            ss_weights_matrix1[i, i] = 1
        row_sums = ss_weights_matrix1.sum(axis=1)
        ss_weights_matrix1 = np.divide(ss_weights_matrix1, row_sums[:, np.newaxis])
        return ss_weights_matrix1

    def generate_walks(self, transition_probs):
        graph = self.graph
        num_walks = self.num_walks
        walk_length = self.walk_length

        walks = []
        nodes = list(graph.nodes())
        probs = {}
        for i, node_i in enumerate(nodes):
            probs[node_i] = {}
            for j, node_j in enumerate(nodes):
                probs[node_i][node_j] = transition_probs[i][j]

        params = []
        for node in nodes:
            params.append((node, num_walks, walk_length, probs))

        pool = mp.Pool()
        node_walks_list = pool.starmap(generate_walks_for_node, params)
        # close the process pool
        pool.close()
        # wait for all tasks to complete
        pool.join()

        for node_walks in node_walks_list:
            walks += node_walks
            
        return walks

    def fit(self, transformation: callable) -> gensim.models.Word2Vec:
        transition_probability_matrix = self.generate_transition(transformation)
        walks = self.generate_walks(transition_probability_matrix)
        return gensim.models.Word2Vec(walks,window=self.window, workers=self.workers, vector_size=self.dimensions)